# Coleta de dados

In [4]:
import pandas as pd
import os
import numpy as np
import yfinance as yf

from typing import Optional, List

import joblib

In [10]:
class AquisicaoDadosFundamentos:
    def __init__(self):
        self.balancos_dir = "dados/balancos/"
        self.balancos_joblib_file = "dados/fundamentos.joblib"
        self.fund_by_code = {}
        self.codes = self.get_code_list()

    def run(self):
        if os.path.isfile(self.balancos_joblib_file):
            print("carregando joblib fundamentos")
            return joblib.load(self.balancos_joblib_file)

        self.get_balancos_by_code()
        self.get_dre_by_code()

        self.salvar_joblib()
        
        return self.fund_by_code
    
    def get_code_list(self):
        return [file.replace("balanco_", "").replace(".xls", "") for file in os.listdir(self.balancos_dir)]

    def get_balancos_by_code(self) -> None:

        files = os.listdir(self.balancos_dir)

        for file in files:
            code = file.replace("balanco_", "").replace(".xls", "")
            print(code)
            balanco = pd.read_excel(f"{self.balancos_dir}{file}", sheet_name=0)
            # colocar codigo na posicao 0, 0
            balanco.iloc[0, 0] = code
            # mudar coluna
            balanco.columns = balanco.iloc[0]
            balanco = balanco[1:]
            # tornar a 1ª coluna (que agora tem o nome da empresa)
            balanco = balanco.set_index(code)
            self.fund_by_code[code] = balanco

    def get_dre_by_code(self) -> None:

        files = os.listdir(self.balancos_dir)
        for file in files:
            code = file.replace("balanco_", "").replace(".xls", "")
            dre = pd.read_excel(f"{self.balancos_dir}{file}", sheet_name=1)
            # na primeira coluna colocar o título com o nome da empresa
            dre.iloc[0, 0] = code
            # pegar 1ª linha e tornar um cabeçalho
            dre.columns = dre.iloc[0]
            dre = dre[1:]
            # tornar a 1ª coluna (que agora tem o nome da empresa)
            dre = dre.set_index(code)
            self.fund_by_code[code] = self.fund_by_code[code].append(dre)
    
    def salvar_joblib(self):
        joblib.dump(self.fund_by_code, self.balancos_joblib_file)


In [12]:
fundamentos_by_code = AquisicaoDadosFundamentos().run()

carregando joblib fundamentos


In [16]:
fundamentos_by_code["ABEV3"].info()

<class 'pandas.core.frame.DataFrame'>
Index: 79 entries, Ativo Total to Lucro/Prejuízo do Período
Data columns (total 33 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   31/12/2020  72 non-null     object
 1   30/09/2020  72 non-null     object
 2   30/06/2020  72 non-null     object
 3   31/03/2020  72 non-null     object
 4   31/12/2019  72 non-null     object
 5   30/09/2019  72 non-null     object
 6   30/06/2019  72 non-null     object
 7   31/03/2019  72 non-null     object
 8   31/12/2018  72 non-null     object
 9   30/09/2018  72 non-null     object
 10  30/06/2018  72 non-null     object
 11  31/03/2018  72 non-null     object
 12  31/12/2017  72 non-null     object
 13  30/09/2017  72 non-null     object
 14  30/06/2017  72 non-null     object
 15  31/03/2017  72 non-null     object
 16  31/12/2016  72 non-null     object
 17  30/09/2016  72 non-null     object
 18  30/06/2016  72 non-null     object
 19  31/03/2016  72 non-null 

In [17]:
def get_ibov_from_yahoo():
    
    ibov_file = "../dados/cotacoes_ibov.joblib"
    # Caso os dados ja tenham sido carregados anteriormente, retorne o joblib
    if os.path.isfile(ibov_file):
        print("carregando joblib ibov")
        return joblib.load(ibov_file)

    data_inicial = "2012-12-20"
    data_final = "2021-09-20"

    ibov = yf.download("^BVSP", start=data_inicial, end=data_final)

    # Salvar arquivo joblib para nao ter que rodar yfinance novamente     
    joblib.dump(ibov, ibov_file)

    return ibov

In [ ]:
def get_cotacoes_from_yahoo(codigos_acoes: List[str]):

    arquivo_cotacoes = "dados/Cotacoes.xlsx"
    # Caso os dados ja tenham sido carregados anteriormente, retorne o joblib
    if os.path.isfile(arquivo_cotacoes):
        cotacoes = pd.read_excel(arquivo_cotacoes, sheet_name=0):
        return cotacoes

    data_inicial = "2012-12-20"
    data_final = "2021-09-20"

    cotacoes_yf = yf.download(codigos_acoes, start=data_inicial, end=data_final)
    
    cotacoes_yf.to_excel(cotacoes_yf, arquivo_cotacoes)
    
    return cotacoes_yf

In [19]:
balancos_dir = "dados/balancos/"
fund_by_code = {}
files = os.listdir(balancos_dir)

for file in files:
    code = file.replace("balanco_", "").replace(".xls", "")
    balanco = pd.read_excel(f"{balancos_dir}{file}", sheet_name=0)
    # colocar codigo na posicao 0, 0
    balanco.iloc[0, 0] = code
    # mudar coluna
    balanco.columns = balanco.iloc[0]
    balanco = balanco[1:]
    # tornar a 1ª coluna (que agora tem o nome da empresa)
    balanco = balanco.set_index(code)
    fund_by_code[code] = balanco


WARNING *** file size (66667) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (64284) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (64282) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (24947) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (30309) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (64136) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (64280) not 512 + multiple of sector size (512)

WARNING *** file size (62367) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (30325) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [ ]:
balancos_dir = "dados/balancos/"
fund_by_code = {}
files = os.listdir(balancos_dir)
for file in files:
    code = file.replace("balanco_", "").replace(".xls", "")
    dre = pd.read_excel(f"{balancos_dir}{file}", sheet_name=1)
    # na primeira coluna colocar o título com o nome da empresa
    dre.iloc[0, 0] = code
    # pegar 1ª linha e tornar um cabeçalho
    dre.columns = dre.iloc[0]
    dre = dre[1:]
    # tornar a 1ª coluna (que agora tem o nome da empresa)
    dre = dre.set_index(code)
    fund_by_code[code] = fund_by_code[code].append(dre)

In [ ]:
def remover_empresas_sem_dados_cotacao(fundamentos_by_code, cotacoes_by_code):
    fundamentos = fundamentos_by_code.copy()
    
    # remove fundamentos das empresas que tenham cotacoes com dados nulos
    
    codes_to_be_removed_from_fund = list(set(fundamentos.keys()) ^ set(cotacoes_by_code.keys()))
    
    for code in codes_to_be_removed_from_fund:
        if code in fundamentos.keys():
            fundamentos.pop(code)

    if cotacoes_by_code.keys() == fundamentos.keys():
        print("Empresas sem dados de cotacoes removidos com sucesso")
    
    return fundamentos

In [ ]:
def remover_empresa_colunas_diff(fundamentos_by_code):

    fundamentos = fundamentos_by_code.copy()
    tamanho_inicial = len(fundamentos)

    columns_ref = list(fundamentos["PETR4"].columns)
    """ 
    Remove empresas que nao tenham colunas de acordo com colunas da empresa referencia 
    """
    codes = fundamentos.keys()
    empresa_a_remover = []
    for code in codes:
        if set(columns_ref) != set(fundamentos[code].columns):
            empresa_a_remover.append(code)
    
    for empresa in empresa_a_remover:
        fundamentos.pop(empresa)
    
    print((tamanho_inicial - len(fundamentos)), "empresas removidas por terem colunas diferentes de PETR4")

    return fundamentos

In [ ]:
def remover_colunas_nulas(fundamentos_by_code):
    fundamentos = fundamentos_by_code.copy()
    codes = fundamentos.keys()

    colunas_remover = ["Receita Bruta de Vendas e/ou Serviços",
                        "Ativos Biológicos",
                        "Despesas Antecipadas",
                        "Deduções da Receita Bruta",
                        "Resultado Não Operacional",
                        "Receitas",
                        "Despesas",
                        "Diferido",
                        "Adiantamento para Futuro Aumento Capital",
                        "Passivos sobre Ativos Não-Correntes a Venda e Descontinuados",
                        "Lucros e Receitas a Apropriar",
                        "Reservas de Reavaliação",
                        "Adiantamento para Futuro Aumento Capital",
                        "Perdas pela Não Recuperabilidade de Ativos",
                        "Outras Receitas Operacionais",
                        "Participações/Contribuições Estatutárias",
                        "Reversão dos Juros sobre Capital Próprio"]
    for code in codes:
        try:
            for col in colunas_remover:
                # verifica se a coluna existe no df antes de executar drop
                if col in fundamentos[code].columns:
                    fundamentos[code].drop(columns=col, inplace=True)
            print(f"colunas nulas de {code} removidas com sucesso")
        except KeyError as e:
            print(f"empresa {code} sem as colunas a remover - err", code)
            raise e

    return fundamentos

In [ ]:
def merge_ibov_by_fundamentos_dates(fundamentos_by_code, ibov):
    fundamentos = fundamentos_by_code.copy()
    datas_fundamentos = fundamentos["PETR4"].index

    # Set as nan when dates are different
    for data in datas_fundamentos:
        if data not in ibov.index:
            ibov.loc[data] = np.nan
    ibov = ibov.sort_index()
    ibov = ibov.ffill()
    ibov = ibov.rename(columns={"Adj Close": "IBOV"})

    for code in fundamentos:
        fundamentos[code] = fundamentos[code].merge(ibov[["IBOV"]], left_index=True, right_index=True)

    print(f"DF fundamentos e ibov juntados com sucesso - tamanho: {len(fundamentos)}")
    return fundamentos

In [ ]:
def juntar_fundamentos_com_cotacoes(fundamentos_by_code, cotacoes_by_code):
    fundamentos = fundamentos_by_code.copy()
    codes = fundamentos.keys()
    for code in codes:
        df = fundamentos[code].T
        df.index = pd.to_datetime(df.index, format="%d/%m/%Y")
        # print(df)

        # Definir data como indice e pegar somente coluna de Adj Close do df
        if code in cotacoes_by_code.keys():
            df_cotacao = cotacoes_by_code[code].set_index("Date")
            df_cotacao = df_cotacao[["Adj Close"]]

            # Juntar dois dataframes
            df = df.merge(df_cotacao, right_index=True, left_index=True)
            # Atualiza o index
            df.index.name = code
            
            fundamentos[code] = df
    print("dataframes juntados com sucesso - ", len(fundamentos))
    return fundamentos

In [ ]:
# exemplo dicionario
fundamentos_by_code = {
    "ABEV3": df,
    "PETR4": df,
}

In [ ]:
def criar_coluna_decisao(dfs_in): 
    
    dfs = dfs_in.copy()

    for code in dfs.keys():
        df = dfs[code]
        
        df = df.sort_index()

        df["cotacao_var"] = df["Adj Close"].shift(-1) / df["Adj Close"] - 1
        df["IBOV_var"] = df["IBOV"].shift(-1) / df["IBOV"] - 1
        df["resultado"] = df["cotacao_var"] - df["IBOV_var"]

        condicoes = [
            (df["resultado"] > 0 ), 
            (df["resultado"] < -0.02)
        ]
        valores = [1, 0]

        df["decisao"] = np.select(condicoes, valores)

        df = df.drop(["Adj Close", "cotacao_var", "IBOV", "IBOV_var", "resultado"], axis=1)

        dfs[code] = df

    print("coluna decisao/target criada com sucesso")

    return dfs